In [1]:
from pyspark.sql import SparkSession 

In [2]:
spark = SparkSession .\
builder .\
appName("spark") .\
config("spark.sql.warehouse.dir","?user/itv016843/warehouse") .\
enableHiveSupport() .\
master("yarn") .\
getOrCreate()

In [3]:
spark

In [4]:
from pyspark.sql.functions import *

In [5]:
from pyspark.sql import  *

In [6]:
! hadoop fs -ls /public/trendytech/lendingclubproject/raw

Found 4 items
drwxr-xr-x   - itv005857 supergroup          0 2023-09-15 14:40 /public/trendytech/lendingclubproject/raw/customers_data_csv
drwxr-xr-x   - itv005857 supergroup          0 2023-09-17 22:57 /public/trendytech/lendingclubproject/raw/loans_data_csv
drwxr-xr-x   - itv005857 supergroup          0 2023-09-18 07:32 /public/trendytech/lendingclubproject/raw/loans_defaulters_csv
drwxr-xr-x   - itv005857 supergroup          0 2023-09-18 07:31 /public/trendytech/lendingclubproject/raw/loans_repayments_csv


In [7]:
delinq_schema = 'member_id string,delinq_2yrs float,delinq_amnt float,pub_rec float,pub_rec_bankruptcies float,inq_last_6mths float,total_rec_late_fee float,mths_since_last_delinq float,mths_since_last_record float'

In [8]:
df1 = spark.read \
.format("csv") \
.option("header",True) \
.schema(delinq_schema) \
.load("/public/trendytech/lendingclubproject/raw/loans_defaulters_csv")

In [9]:
df1.limit(5)

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record
673da0da820de70f9...,0.0,0.0,0.0,0.0,1.0,0.0,33.0,null
f5725a82fea842f9b...,0.0,0.0,0.0,0.0,2.0,0.0,49.0,null
0e697722917e34328...,0.0,0.0,0.0,0.0,2.0,0.0,33.0,null
e995ef7b48b9a42db...,0.0,0.0,0.0,0.0,1.0,0.0,null,null
a3b9c8c1aaa89b52a...,0.0,0.0,0.0,0.0,1.0,0.0,45.0,null


In [10]:
df1.groupBy("delinq_2yrs").count()

delinq_2yrs,count
20.04,1
18.53,1
18.0,30
26.24,1
6.52,1
9.0,818
21.72,1
17.17,1
58.0,1
5.0,6599


In [11]:
df1.groupBy("delinq_2yrs").count().limit(5)

delinq_2yrs,count
20.04,1
18.53,1
26.24,1
6.52,1
17.17,1


In [12]:
df2 = df1.withColumn("Time",current_timestamp())

In [13]:
df2.limit(5)

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record,Time
673da0da820de70f9...,0.0,0.0,0.0,0.0,1.0,0.0,33.0,null,2025-07-26 05:04:...
f5725a82fea842f9b...,0.0,0.0,0.0,0.0,2.0,0.0,49.0,null,2025-07-26 05:04:...
0e697722917e34328...,0.0,0.0,0.0,0.0,2.0,0.0,33.0,null,2025-07-26 05:04:...
e995ef7b48b9a42db...,0.0,0.0,0.0,0.0,1.0,0.0,null,null,2025-07-26 05:04:...
a3b9c8c1aaa89b52a...,0.0,0.0,0.0,0.0,1.0,0.0,45.0,null,2025-07-26 05:04:...


In [14]:
df3 = df2.withColumn("delinq_2yrs",col("delinq_2yrs").cast("int"))

In [16]:
df4 = df3.withColumn("delinq_amnt",col("delinq_amnt").cast("int"))

In [17]:
df4.limit(5)

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record,Time
673da0da820de70f9...,0,0,0.0,0.0,1.0,0.0,33.0,null,2025-07-26 05:05:...
f5725a82fea842f9b...,0,0,0.0,0.0,2.0,0.0,49.0,null,2025-07-26 05:05:...
0e697722917e34328...,0,0,0.0,0.0,2.0,0.0,33.0,null,2025-07-26 05:05:...
e995ef7b48b9a42db...,0,0,0.0,0.0,1.0,0.0,null,null,2025-07-26 05:05:...
a3b9c8c1aaa89b52a...,0,0,0.0,0.0,1.0,0.0,45.0,null,2025-07-26 05:05:...


In [19]:
df5 = df4.withColumn("mths_since_last_delinq",col("mths_since_last_delinq").cast("int"))

In [20]:
df5.limit(5)

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record,Time
673da0da820de70f9...,0,0,0.0,0.0,1.0,0.0,33,null,2025-07-26 05:06:...
f5725a82fea842f9b...,0,0,0.0,0.0,2.0,0.0,49,null,2025-07-26 05:06:...
0e697722917e34328...,0,0,0.0,0.0,2.0,0.0,33,null,2025-07-26 05:06:...
e995ef7b48b9a42db...,0,0,0.0,0.0,1.0,0.0,null,null,2025-07-26 05:06:...
a3b9c8c1aaa89b52a...,0,0,0.0,0.0,1.0,0.0,45,null,2025-07-26 05:06:...


In [21]:
df5.createOrReplaceTempView("data")

In [22]:
spark.sql("use ajdb1")

""


In [27]:
df6 = spark.sql("select member_id,delinq_2yrs,delinq_amnt,mths_since_last_delinq from data where delinq_2yrs > 0") 

In [28]:
df6.write \
.format("csv") \
.mode("overwrite") \
.option("header",True) \
.option("path","/user/itv016843/cleaned/delinq") \
.save()

In [29]:
df5.limit(5)

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record,Time
673da0da820de70f9...,0,0,0.0,0.0,1.0,0.0,33,null,2025-07-26 05:12:...
f5725a82fea842f9b...,0,0,0.0,0.0,2.0,0.0,49,null,2025-07-26 05:12:...
0e697722917e34328...,0,0,0.0,0.0,2.0,0.0,33,null,2025-07-26 05:12:...
e995ef7b48b9a42db...,0,0,0.0,0.0,1.0,0.0,null,null,2025-07-26 05:12:...
a3b9c8c1aaa89b52a...,0,0,0.0,0.0,1.0,0.0,45,null,2025-07-26 05:12:...


In [34]:
df7 = spark.sql("select member_id,int(pub_rec),int(pub_rec_bankruptcies),int(inq_last_6mths) from data where pub_rec_bankruptcies > 0")

In [35]:
df7

member_id,pub_rec,pub_rec_bankruptcies,inq_last_6mths
0dd2bbc517e3c8f9e...,1,1,3
458458599d3df3bfc...,1,1,1
09a1c6855801dad88...,1,1,0
56d4375718ad6940d...,1,1,0
41a307b3c2c66aad8...,1,1,0
3a4aac093cddcdb0c...,1,1,0
d0e339cf3578b24e9...,1,1,0
4c5cc017a06ad9cee...,1,1,0
7d746a6d68e460263...,1,1,2
c6729bc4f914f5c5f...,1,1,0


In [ ]:
df7.write \
.format("csv") \
.mode("overwrite") \
.option("header",True) \
.